# Análisis exploratorio de datos y extracción de características con Python

Usando **visualización de datos**, **ingeniería de funciones** y **selección de funciones** para hacer que una simple **regresión logística** parezca poderosa.

Yo era un niño cuando se estrenó la película 'Titanic'. En mi mente aún guardo una imagen de toda esa gente, afuera del cine, en la fila para comprar boletos. En ese entonces, Leonardo DiCaprio y Kate Winslet eran solo dos niños pequeños con lindos cortes de cabello y los boletos en línea eran ciencia ficción.
​
Según los ingenieros, clase a la que orgullosamente pertenezco, el Titanic era el barco insumergible. Era hermoso, lujoso y equipado con lo mejor de la tecnología. Titanic fue el crucero de última generación. Simplemente no era tan insumergible.
​
Después de 100 años, el Titanic sigue siendo tema de discusión en los más diversos ámbitos. Por ejemplo, puede encontrar [libros](https://amzn.to/2Gie0Pv) donde el autor toma lecciones de liderazgo del Titanic para aplicar en los negocios; puede encontrar [proyectos de IA] interesantes (http://fortune.com/2018/03/26/china-titanic-artificial-intelligence-sensetime/) que aplican el aprendizaje profundo para distinguir las escenas románticas del Titanic de las escenas de desastre; o puede encontrar extensos ejercicios de pensamiento creativo sobre [lo que realmente sucedió con el Titanic](https://www.bustle.com/p/6-titanic-conspiracy-theories-that-are-still-fascinating-today-28519 ).
​
También usaremos Titanic para un propósito específico: aprender técnicas de **análisis exploratorio de datos** y **extracción de características**. A través de un análisis completo del problema del Titanic de Kaggle, veremos qué hacer, por dónde empezar y cómo proceder en un problema de ciencia de datos. Se abordarán temas como **visualización de datos**, **imputación de datos faltantes**, **ingeniería de funciones**, **selección de funciones** y **regresión logística**, que le servirán repetidamente porque después de ver lo que involucrado, podrá aplicar estas técnicas a cualquier tipo de problema de ciencia de datos.
​
Dado que soy un verdadero creyente del poder transformador de las empresas emergentes y veo similitudes obvias entre lo que hacen las empresas emergentes y lo que hace un científico de datos, varias referencias a los métodos de empresas emergentes están presentes en el texto. Además, encontrarás algunos chistes sobre el Titanic. No es que crea en su poder para hacer reír a alguien, pero solo porque son geniales para romper el hielo...
​
Este núcleo se ha dividido en cuatro partes. La primera parte trata sobre el desarrollo de un modelo de referencia. Este modelo debería permitirnos comprender rápidamente el problema y los datos. Después, entraremos en detalles. Los datos se estudiarán y enriquecerán a través del análisis exploratorio de datos y la extracción de características, para mejorar el rendimiento de nuestro modelo de aprendizaje automático. Finalmente, se extraerán algunas conclusiones de este kernel y su impacto en nuestro viaje de ciencia de datos.
​
**Índice**
​
1. [El conjunto de datos lean](#1.-The-lean-data-set)
​
  1.1. [Hacer el lanzamiento] (#1.1.-Hacer el lanzamiento)
 
  1.2. [Mostrando los números](#1.2.-Mostrando-los-números)
 
  1.3. [Rellenando los huecos](#1.3.-Rellenando-los-huecos)
 
  1.4. [Modelo mínimo viable](#1.4.-Modelo-mínimo-viable)
 
2. [El conjunto de datos gorditos](#2.-El-conjunto-de-datos-gorditos)
 
  2.1. [Imputación de datos faltantes de 'Edad'](#2.1.-Imputación-de-datos-faltantes-de-'Edad')
 
  2.2. [Análisis exploratorio de datos](#2.2.-Análisis-exploratorio-de-datos)
 
  2.3. [Extracción de características](#2.3.-Extracción de características)
 
3. [Modelo Unicornio](#3.-Modelo Unicornio)
​
  3.1. [Ajustar modelo para la mejor combinación de características](#3.1.-Ajustar-modelo-para-la-mejor-combinación-de-características)
 
  3.2. [Curva de aprendizaje](#3.2.-Curva-de-aprendizaje)
 
  3.3. [Curva de validación](#3.3.-Curva-de-validación)
 
  3.4. [Enviar predicciones](#3.4.-Enviar-predicciones)
 
4. [Conclusión](#4.-Conclusión)
​
**Advertencia:** Esta será una lectura larga. Prepararse. Traiga su cuaderno, siéntese en su silla favorita y vierta Cola en un vaso lleno de hielo. El hielo es importante porque la cola siempre va bien con hielo. Igual que el Titanic (te lo dije).
​
---

#0. Belfast, una incubadora anterior

Las incubadoras son empresas que apoyan la creación de startups y sus primeros años de actividad. Son importantes porque ayudan a los empresarios a resolver algunos problemas comúnmente asociados con la gestión de un negocio, como el espacio de trabajo, la capacitación y la financiación inicial.

Nuestra obra maestra de ingeniería también necesita un punto de partida. En esta sección, comenzamos el ensamblaje de nuestro trabajo importando algunas bibliotecas y funciones generales.

## Imports

In [ ]:
# Import librerias
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# Pon esto cuando se llame
from sklearn.model_selection import train_test_split
from sklearn.model_selection import learning_curve
from sklearn.model_selection import validation_curve
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

## Functions

In [ ]:
# Crear tabla para análisis de datos faltantes
def draw_missing_data_table(df):
    total = df.isnull().sum().sort_values(ascending=False)
    percent = (df.isnull().sum()/df.isnull().count()).sort_values(ascending=False)
    missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    return missing_data

In [ ]:
# Trazar la curva de aprendizaje
def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
                        n_jobs=1, train_sizes=np.linspace(.1, 1.0, 5)):
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Validation score")

    plt.legend(loc="best")
    return plt

In [ ]:
# Trazar la curva de validación
def plot_validation_curve(estimator, title, X, y, param_name, param_range, ylim=None, cv=None,
                        n_jobs=1, train_sizes=np.linspace(.1, 1.0, 5)):
    train_scores, test_scores = validation_curve(estimator, X, y, param_name, param_range, cv)
    train_mean = np.mean(train_scores, axis=1)
    train_std = np.std(train_scores, axis=1)
    test_mean = np.mean(test_scores, axis=1)
    test_std = np.std(test_scores, axis=1)
    plt.plot(param_range, train_mean, color='r', marker='o', markersize=5, label='Training score')
    plt.fill_between(param_range, train_mean + train_std, train_mean - train_std, alpha=0.15, color='r')
    plt.plot(param_range, test_mean, color='g', linestyle='--', marker='s', markersize=5, label='Validation score')
    plt.fill_between(param_range, test_mean + test_std, test_mean - test_std, alpha=0.15, color='g')
    plt.grid() 
    plt.xscale('log')
    plt.legend(loc='best') 
    plt.xlabel('Parameter') 
    plt.ylabel('Score') 
    plt.ylim(ylim)

---

# 1. El conjunto de datos de aprendizaje

En el libro ['The Lean Startup'](https://amzn.to/2sHpnvP), Eric Ries nos cuenta sus experiencias personales adaptando los principios de gestión lean a empresas emergentes de alta tecnología. A través de una serie de anécdotas e historias, Ries nos enseña todo lo que debemos saber sobre la agilidad y la metodología lean en el mundo de las startups.

Si bien se enseña un conjunto de principios importantes a lo largo del libro, la verdad es que la metodología lean startup siempre termina en un intento de responder a la pregunta: '¿Debe construirse este producto?'

Para responder a esta pregunta, el enfoque lean startup se basa en un proceso Construir-Medir-Aprender. Este proceso enfatiza la iteración rápida como un ingrediente crítico para el desarrollo de productos. Pasa por las siguientes fases:
1. **Construir**. Averigüe el problema que debe resolverse, genere ideas sobre cómo resolverlo y seleccione la mejor. Convierte tu mejor idea en un Producto Mínimo Viable (MVP).
2. **Medir**. Pruebe su producto. Dirígete a tus clientes y mide sus reacciones y comportamientos frente a tu producto.
3. **Aprender**. Analice los datos que recopiló al probar el producto con sus clientes. Sacar conclusiones del experimento y decidir qué hacer a continuación.
En otras palabras, este es un proceso de aprendizaje validado que crea, prueba y reconstruye productos rápidamente, de acuerdo con los comentarios de los usuarios. Esto reduce sus riesgos de mercado al fallar rápido y barato, para acercarlo más y más a lo que el mercado realmente necesita.

Este núcleo hace algo similar. Intentaremos fallar de forma rápida y económica construyendo rápidamente una canalización de extremo a extremo que funcione (Build). Luego, instrumentaremos el sistema para evaluar su desempeño (Medir). Finalmente, haremos cambios incrementales para mejorar el rendimiento del sistema (Learn). Tenga en cuenta que esta metodología práctica fue adaptada de Goodfellow et al. (2016), un libro al que puede acceder de forma gratuita [aquí] (http://www.deeplearningbook.org/).

Inicialmente, no invertiremos mucho tiempo en el análisis exploratorio de datos. Simplemente haremos el mínimo esfuerzo viable para implementar un modelo razonable. Este modelo será nuestro 'Modelo Mínimo Viable'. Más adelante, intentaremos superar este modelo enriqueciendo nuestros datos.

**Pregunta sorpresa:** ¿fue el Titanic un MVP?

## 1.1. haciendo el lanzamiento

Las startups usan lanzamientos para vender su idea. En consecuencia, su presentación debe ser clara y concisa, respondiendo a preguntas como '¿qué haces?', '¿qué quieres?' y '¿quién está en tu equipo?'. El tono es importante porque los inversores están más dispuestos a invertir cuando entienden lo que estás haciendo.

Devolvamos las primeras filas de nuestro conjunto de datos para obtener una imagen clara y concisa de lo que hay allí y lo que podemos hacer con él.

In [ ]:
# Importando datos
df = pd.read_csv('../input/train.csv')
df_raw = df.copy()  # Save original data set, just in case.

In [ ]:
# Descripción general
df.head()

Definiciones y pensamientos rápidos:

* **Id del Pasajero**. Identificación única del pasajero. No debería ser necesario para el modelo de aprendizaje automático.
* **Sobrevivió**. Supervivencia (0 = No, 1 = Sí). Variable binaria que será nuestra variable objetivo.
* **Clase P**. Clase de boleto (1 = 1°, 2 = 2°, 3 = 3°). Listo para ir.
* **Nombre**. Nombre del pasajero. Necesitamos analizar antes de usarlo.
* **Sexo**. Sexo. Variable categórica que debe [codificarse] (http://scikit-learn.org/stable/modules/preprocessing.html#preprocessing-categorical-features).
* **Edad**. Edad en años. Listo para ir.
* **Esp.Sib**. # de hermanos/cónyuges a bordo del Titanic. Listo para ir.
* **Parque**. # de padres/hijos a bordo del Titanic. Listo para ir.
* **Boleto**. Numero de ticket. Gran desorden. Necesitamos entender su estructura primero.
* **Tarifa**. Tarifa de pasajero. Listo para ir.
* **Cabina**. Número de cabina. Necesita ser analizado.
* **Embarcado**. Puerto de embarque (C = Cherburgo, Q = Queenstown, S = Southampton). Rasgo categórico que debe ser codificado.

La conclusión principal es que ya tenemos un conjunto de características que podemos usar fácilmente en nuestro modelo de aprendizaje automático. Otras características, como 'Nombre', 'Boleto' y 'Tarifa', requieren un esfuerzo adicional antes de que podamos integrarlas.

## 1.2. Mostrando los números

Los números son cruciales para establecer objetivos, tomar decisiones comerciales acertadas y obtener dinero de los inversores. Con números puedes proyectar el futuro de tu startup, para que todos puedan entender cuáles son las expectativas en torno a tu idea.

Del mismo modo, generaremos las estadísticas descriptivas para obtener la información cuantitativa básica sobre las características de nuestro conjunto de datos.

In [ ]:
# Estadísticas descriptivas
df.describe()

Hay tres aspectos que suelen llamar mi atención cuando analizo estadísticas descriptivas:
1. **Valores mínimos y máximos**. Esto puede darnos una idea sobre el rango de valores y es útil para detectar valores atípicos. En nuestro caso, todos los valores mínimos y máximos parecen razonables y dentro de un rango razonable de valores. La única excepción podría ser el valor máximo de 'Tarifa', pero por ahora lo dejaremos como está.
2. **Media y desviación estándar**. La media nos muestra la tendencia central de la distribución, mientras que la desviación estándar cuantifica su cantidad de variación. Por ejemplo, una desviación estándar baja sugiere que los puntos de datos tienden a estar cerca de la media. Dando un vistazo rápido a nuestros valores, no hay nada que parezca obviamente incorrecto.
3. **Contar**. Esto es importante para darnos una primera percepción sobre el volumen de datos faltantes. Aquí, podemos ver que faltan algunos datos de 'Edad'.

Dado que no hay nada impactante en las variables, pasemos al siguiente paso: datos faltantes.

## 1.3. llenando los huecos

Una de mis definiciones favoritas de startup pertenece a Eric Ries: 'una startup es una institución humana diseñada para crear un nuevo producto o servicio en condiciones de extrema incertidumbre'.

La palabra 'incertidumbre' es clave en esta definición y también es clave en la falta de datos. Los datos faltantes ocurren cuando no hay disponible ningún valor de datos en una o más variables. En consecuencia, reduce el tamaño del conjunto de datos y es una posible fuente de sesgo, ya que algún mecanismo no aleatorio puede estar generando los datos que faltan. Como resultado, la falta de datos introduce incertidumbre en nuestro análisis.

Existen varias estrategias para lidiar con los datos faltantes. Algunos de los más comunes son:
* Utilizar solo datos válidos, eliminando los casos en los que falten datos.
* Imputa datos usando valores de casos similares o usando el valor medio.
* Imputar datos utilizando métodos basados en modelos, en los que se definen modelos para predecir los valores que faltan.

Hasta hoy, nunca he encontrado una solución de "talla única". Tengo algunos dogmas (por ejemplo, suelo excluir variables con más del 25% de datos faltantes), pero lo que suele guiar mi análisis es la intuición, el pensamiento crítico y la necesidad (a veces necesitamos dejar nuestros dogmas en la puerta, si queremos generar algunos resultados).

Mi consejo práctico para manejar los datos faltantes es aprender un conjunto diferente de herramientas. Juega con ellos según tus necesidades, pruébalos y deberías estar bien. Una buena introducción al tema se puede encontrar en [Hair et al. (2013)](https://amzn.to/2M9v0uW). Este libro tiene un resumen práctico sobre los datos que faltan y proporciona un marco que puede aplicar en casi todas las situaciones. Además, escribí un documento técnico comparando diferentes técnicas de imputación, que puedo compartir con usted si lo desea.

Ahora que podemos ver la punta del iceberg, profundicemos en el tema.

In [ ]:
# Analizar datos faltantes
draw_missing_data_table(df)

Primeros pensamientos:

* 'Cabina' tiene demasiados valores faltantes (>25%). ¡Dogma! Necesitamos eliminar esta variable de inmediato.
* Se puede imputar 'Edad'. Por ahora, asociaré un valor que me permita saber que estoy imputando datos. Más tarde, revisaré esta estrategia.
* Debido al bajo porcentaje de valores faltantes, eliminaré las observaciones donde no sabemos 'Embarcado'.

In [ ]:
# Caída de cabina
df.drop('Cabin', axis=1, inplace=True)
df.head()

In [ ]:
# Rellene los valores faltantes en Edad con un valor específico
value = 1000
df['Age'].fillna(1000, inplace=True)
df['Age'].max()

In [ ]:
# Borrar observaciones sin Embarcado
df.drop(df[pd.isnull(df['Embarked'])].index, inplace=True)  # Obtener índice de puntos donde Embarked es nulo
df[pd.isnull(df['Embarked'])]

## 1.4. Modelo mínimo viable

El 'Producto Mínimo Viable' (MVP) es un concepto clave para cualquier startup lean. Una vez que se resuelve el problema a resolver, el enfoque de la puesta en marcha debe estar en el desarrollo de una solución, el MVP, lo más rápido posible. Gracias al MVP, es posible iniciar el proceso de aprendizaje y mejorar la solución hacia las necesidades de los usuarios.

[Goodfellow et al. (2016)](http://www.deeplearningbook.org/contents/guidelines.html) propone un enfoque análogo para la aplicación de modelos de aprendizaje automático. Como señalan los autores, la aplicación exitosa de técnicas de aprendizaje automático va más allá del conocimiento de los algoritmos y sus principios. Para aplicar con éxito las técnicas de aprendizaje automático, debemos comenzar con un modelo simple que podamos dominar y comprender. Solo entonces deberíamos pasar a algoritmos más complejos.

Los autores proponen una metodología práctica de cuatro pasos:
1. Seleccione una métrica de rendimiento y un valor objetivo para esta métrica. Esta métrica guiará su trabajo y le permitirá saber qué tan bien se está desempeñando. En nuestro caso, nuestra métrica de rendimiento será la "precisión" porque es la definida por [Kaggle](https://www.kaggle.com/c/titanic#e Evaluation).
2. Configure rápidamente una canalización integral que funcione. Esto debería permitirle estimar la métrica de rendimiento seleccionada.
3. Supervisar el sistema para comprender su comportamiento, en particular para comprender si su bajo rendimiento está relacionado con un ajuste insuficiente, un ajuste excesivo o defectos.
4. Mejorar el sistema por iteración. Aquí podemos aplicar ingeniería de características, ajustar hiperparámetros o incluso cambiar el algoritmo, de acuerdo con las salidas de nuestro sistema de monitoreo.

Seguiremos esta metodología. En consecuencia, nuestro objetivo será obtener un modelo inicial que podamos utilizar como un primer enfoque de referencia. Este modelo será nuestro 'Modelo Mínimo Viable' (MVM). Tenga en cuenta que en este momento no importa mucho qué tan bien se desempeñe el modelo. Sólo necesitamos un punto de partida. En definitiva, somos empresarios. En el peor de los casos, llamamos a este modelo 'versión beta' :P

Bien, preparemos los datos para el lanzamiento de MVM, ajustemos una regresión logística y analicemos el rendimiento del modelo a través de [curvas de aprendizaje y validación] (http://scikit-learn.org/stable/modules/learning_curve.html ).

### 1.4.1. Preparando los datos

In [ ]:
# Tipos de datos
df.dtypes

* No necesitamos 'PassengerId' para fines de predicción, por lo que lo excluiremos.
* 'Sexo', 'Embarcado' y 'Pclase' deben ser categóricos. No consideraré 'Sobrevivido' como categórico porque es la variable de salida.
* Necesitamos analizar 'Nombre' y 'Boleto'. Por ahora, ignoraré estas características.
* 'SibSp' podría agruparse con 'Parch' para crear una característica de 'Familia'. Por ahora solo identificaré si el pasajero viaja solo o con su familia.

In [ ]:
# Soltar ID de pasajero
df.drop('PassengerId', axis=1, inplace=True)
df.head()

In [ ]:
# Definir variables categóricas
df['Sex'] = pd.Categorical(df['Sex'])
df['Embarked'] = pd.Categorical(df['Embarked'])

In [ ]:
# Crear función familiar
df['FamilySize'] = df['SibSp'] + df['Parch']
df.head()

In [ ]:
# Soltar SibSp y Parch
df.drop('SibSp',axis=1,inplace=True)
df.drop('Parch',axis=1,inplace=True)
df.head()

In [ ]:
# Soltar Nombre y Ticket
df.drop('Name', axis=1, inplace=True)
df.drop('Ticket', axis=1, inplace=True)
df.head()

### 1.4.2. Lanzamiento del modelo

Ready... Set... [Go!](https://youtu.be/_YzNZE287QQ)

In [ ]:
# Transformar variables categóricas en variables ficticias
df = pd.get_dummies(df, drop_first=True) # Para evitar la trampa ficticia
df.head()

Extra: [¿Qué es la 'trampa de variable ficticia'?](http://www.algosome.com/articles/dummy-variable-trap-regression.html)

In [ ]:
# Crear un conjunto de datos para entrenar métodos de imputación de datos
X = df[df.loc[:, df.columns != 'Survived'].columns]
y = df['Survived']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=1)

In [ ]:
# Debug
print('Inputs: \n', X_train.head())
print('Outputs: \n', y_train.head())

In [ ]:
# Ajustar regresión logística
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

In [ ]:
# Rendimiento del modelo
scores = cross_val_score(logreg, X_train, y_train, cv=10)
print('CV accuracy: %.3f +/- %.3f' % (np.mean(scores), np.std(scores)))

### 1.4.3.  Assessing model performance

In [ ]:
# Trazar curvas de aprendizaje
title = "Learning Curves (Logistic Regression)"
cv = 10
plot_learning_curve(logreg, title, X_train, y_train, ylim=(0.7, 1.01), cv=cv, n_jobs=1);

Curvas de aprendizaje en pocas palabras:
* Las curvas de aprendizaje nos permiten diagnosticar si el **sobreajuste** o **falta de ajuste**.
* Cuando el modelo **sobreajusta**, significa que funciona bien en el conjunto de entrenamiento, pero no en el conjunto de validación. En consecuencia, el modelo no es capaz de generalizar a datos no vistos. Si el modelo se sobreajusta, la curva de aprendizaje presentará una brecha entre los puntajes de entrenamiento y validación. Dos soluciones comunes para el sobreajuste son reducir la complejidad del modelo y/o recopilar más datos.
* Por otro lado, el **ajuste insuficiente** significa que el modelo no puede funcionar bien en los conjuntos de entrenamiento o validación. En esos casos, las curvas de aprendizaje convergerán a un valor de puntaje bajo. Cuando el modelo no se ajusta bien, la recopilación de más datos no es útil porque el modelo ya no puede aprender los datos de entrenamiento. Por lo tanto, los mejores enfoques para estos casos son mejorar el modelo (p. ej., ajustar los hiperparámetros) o mejorar la calidad de los datos (p. ej., recopilar un conjunto diferente de características).
​
Discusión de nuestros resultados:
* El modelo no se sobreencaja. Como podemos ver, las curvas convergen y no existe brecha entre el entrenamiento y la puntuación de validación en los últimos puntos de la curva.
* El modelo se desadapta. Nuestra puntuación final es de aproximadamente 0,786. Aunque nuestro modelo hace mejores predicciones que una [estrategia de lanzar una moneda] (https://en.wikipedia.org/wiki/Flipism), todavía está lejos de ser un modelo 'inteligente'. Por ahora, es solo un modelo 'artificial'.

In [ ]:
# Trazar la curva de validación
title = 'Validation Curve (Logistic Regression)'
param_name = 'C'
param_range = [0.001, 0.01, 0.1, 1.0, 10.0, 100.0] 
cv = 10
plot_validation_curve(estimator=logreg, title=title, X=X_train, y=y_train, param_name=param_name,
                      ylim=(0.5, 1.01), param_range=param_range);

Curvas de validación en pocas palabras:
* Las curvas de validación son una herramienta que podemos utilizar para mejorar el rendimiento de nuestro modelo. Cuenta como una forma de ajustar nuestros hiperparámetros.
* Son diferentes a las curvas de aprendizaje. Aquí, el objetivo es ver cómo el parámetro del modelo afecta los puntajes de capacitación y validación. Esto nos permite elegir un valor diferente para el parámetro, para mejorar el modelo.
* Una vez más, si hay una brecha entre el entrenamiento y el puntaje de validación, es probable que el modelo esté sobreajustado. Por el contrario, si no hay brecha pero el valor de la puntuación es bajo, podemos decir que el modelo no se ajusta.

Discusión de nuestros resultados:
* La figura muestra que no hay una gran diferencia en el rendimiento del modelo en la medida en que elegimos un valor C de $10^{-1}$ o superior. Tenga en cuenta que en una regresión logística, C es el único parámetro del modelo que podemos cambiar [(consulte la documentación de scikit-learn)](http://scikit-learn.org/stable/modules/linear_model.html#logistic-regression). 

---

# 2. El conjunto de datos gordito

En este punto, nuestro modelo:

* Puede lograr una precisión de 0,786 +/- 0,026.
* Se basa en una regresión logística.
* Utiliza 'Pclass', 'Edad', 'Tarifa', 'FamilySize', 'Sex' y 'Embarked como entradas; y 'Sobrevivió' como salida.

Además, en cuanto a la metodología práctica que comentábamos antes, podemos decir que:
1. La elección de la métrica de rendimiento es un tema cerrado porque estamos siguiendo las especificaciones de Kaggle.
2. Nuestro modelo actual puede funcionar como un modelo de referencia y es el resultado de una canalización integral en funcionamiento.
3. Las curvas de aprendizaje y validación nos permiten monitorear el desempeño del sistema.

Como consecuencia, solo falta el cuarto y último paso de la metodología práctica: mejorar el modelo por iteración. Esto se puede hacer por:

* Mejorar la forma en que manejamos los datos faltantes de 'Edad'. En nuestro enfoque lean, decidimos reemplazar los datos faltantes por un valor único, pero ahora podemos profundizar y buscar una mejor estrategia de imputación.
* Exploración de datos para comprender qué características pueden tener un impacto en el modelo y cómo se pueden manipular para impulsar ese impacto.
* Construir nuevas funcionalidades que puedan incrementar el poder predictivo de nuestro modelo.

Esto nos llevará a un proceso pesado de análisis de datos, cuyo objetivo es mejorar el rendimiento del modelo solo por el lado de la calidad de los datos. En otras palabras, no cambiaremos nuestro algoritmo de aprendizaje ni intentaremos mejorar sus parámetros. Solo intentaremos mejorar el rendimiento de nuestro modelo enriqueciendo nuestros datos.

Dicho esto, digamos adiós al enfoque magro y demos la bienvenida al enfoque gordito.

In [ ]:
# Reiniciar conjunto de datos
df = df_raw.copy()
df.head()

In [ ]:
# Función de tamaño familiar
df['FamilySize'] = df['SibSp'] + df['Parch']
df.head()

In [ ]:
# Soltar SibSp y Parch
df.drop('SibSp',axis=1,inplace=True)
df.drop('Parch',axis=1,inplace=True)
df.head()

In [ ]:
# Eliminar características irrelevantes
df.drop(['Name','Ticket','Cabin'], axis=1, inplace=True)
df.head()

## 2.1. Imputación de datos faltantes de 'Edad'

Nuestro enfoque inicial para estimar los valores faltantes de 'Edad' fue llenar con un valor de marcador de posición (1000). Esto nos permitió obtener rápidamente un conjunto de datos completo, en el que era fácil identificar los valores imputados. Dado que nuestro objetivo era tener una tubería de extremo a extremo que funcionara lo más rápido posible, este enfoque estuvo bien. Sin embargo, tiene varias limitaciones. Por ejemplo, estamos usando valores de reemplazo poco realistas, que están fuera de rango y distorsionan la distribución de datos. En consecuencia, ahora que estamos mejorando el modelo, tiene sentido desarrollar un método de imputación diferente.

Una forma de mejorar nuestro método de imputación es estimar los valores faltantes en función de las relaciones conocidas. En nuestro caso, podemos hacer esto usando la información en la variable 'Nombre'. Mirando los valores de 'Nombre', podemos ver el nombre y el título de la persona. El título de la persona es una información relevante para estimar edades. Para dar un ejemplo, sabemos que una persona con el título de 'Maestro' es alguien menor de 13 años, ya que ['un niño puede ser llamado maestro solo hasta los 12 años'](http://bit.ly/2HfFHZr) . Por tanto, empleando la información de 'Nombre' podemos mejorar nuestro método de imputación.

Los pasos para implementar este nuevo método de imputación son:
* Extraer títulos de 'Nombre'.
* Trace una figura con ambas características y confirme que existe una conexión entre los títulos y la edad.
* Para cada título, obtenga la edad promedio de las personas y utilícela para completar los valores faltantes.

Veamos cómo funciona esto, antes de que empieces a hundirte en los sentimientos.

In [ ]:
# Inspeccionar nombres
df_raw['Name'].unique()[:10]

* La regla parece ser: *'apellido'* + *','* + *'título'* + *'otros nombres'*

In [ ]:
# Extraer títulos del nombre
df['Title']=0
for i in df:
    df['Title']=df_raw['Name'].str.extract('([A-Za-z]+)\.', expand=False)  # Use REGEX to define a search pattern
df.head()

In [ ]:
# Títulos únicos
df['Title'].unique()

In [ ]:
# Parcela de barras (títulos, edad y sexo)
plt.figure(figsize=(15,5))
sns.barplot(x=df['Title'], y=df_raw['Age']);

* El gráfico de barras nos da una estimación de la tendencia central de una variable numérica (altura de cada rectángulo) y una indicación de la incertidumbre en torno a esa estimación (barras de error en negro).
* Aparte de Rev y Dr, que tienen una barra de error más grande, el valor medio parece representar con precisión los datos de todas las demás funciones. Esto valida nuestro enfoque.
* [Aquí](https://www.biologyforlife.com/interpreting-error-bars.html) puede encontrar una breve y dulce introducción a la interpretación de las barras de error.

In [ ]:
# Medios por título
df_raw['Title'] = df['Title']  # Para simplificar el manejo de datos
means = df_raw.groupby('Title')['Age'].mean()
means.head()

In [ ]:
# Transform means into a dictionary for future mapping
map_means = means.to_dict()
map_means

In [ ]:
# Impute ages based on titles
idx_nan_age = df.loc[np.isnan(df['Age'])].index
df.loc[idx_nan_age,'Age'].loc[idx_nan_age] = df['Title'].loc[idx_nan_age].map(map_means)
df.head()

In [ ]:
# Identificar datos imputados
df['Imputed'] = 0
df.at[idx_nan_age.values, 'Imputed'] = 1
df.head()

## 2.2. Análisis exploratorio de datos

El análisis exploratorio de datos se menciona a menudo como uno de los pasos más importantes en el proceso de análisis de datos. Sin embargo, es bastante fácil caer en una trampa de 'inmersión de datos' (especialmente si está resolviendo problemas sobre barcos hundidos) y perderse en el proceso. Cuando eso sucede, su análisis puede terminar así [así] (https://youtu.be/CIQI3isddbE).

Podemos evitar esto siguiendo un enfoque basado en hipótesis. El enfoque basado en hipótesis consiste en establecer hipótesis sobre el comportamiento de las variables y sus relaciones, al principio del proceso, para luego centrarse en usar datos para probar (o refutar) esas hipótesis. Esto hace que nuestro análisis sea muy objetivo porque recopilaremos los datos suficientes para probar hipótesis específicas. Como resultado, nosotros:
* Aumentar la velocidad. Ya que limitaremos nuestro análisis a alguna hipótesis y seguiremos adelante.
* Reducir el esfuerzo. La cantidad de datos y el número de pruebas será solo lo necesario para verificar su hipótesis.
* Reducir el riesgo. Si aciertas, triunfas rápido, si te equivocas, fracasas rápido.

[Aquí](http://gsl.mit.edu/media/programs/ghana-summer-2013/materials/problem_solution_grand_slam_7_steps_to_master_training_deck.pdf) puede encontrar una de mis presentaciones de PowerPoint favoritas sobre los beneficios y procedimientos de un enfoque basado en hipótesis en resolución de problemas Tenga en cuenta que, particularmente cuando realmente necesita aprender sobre el conjunto de datos, tiene sentido colocar el cilindro de buceo y sumergirse en las profundidades del análisis de datos. Sin embargo, si al principio puede generar una suposición fundamentada de cuál es la respuesta de su problema, creo que debe probar su hipótesis y aprender de ella lo más rápido que pueda.

Bien, ahora que te convencí de que el enfoque basado en hipótesis es la [última coca cola en el desierto] (http://bit.ly/2oYzJ7U), déjame mostrarte cómo aplicarlo. Casos como el que tenemos son objetivos fáciles para el enfoque basado en hipótesis porque no tenemos muchas variables, por lo que podemos adivinar más o menos su impacto. En consecuencia, comenzaremos enumerando cada una de las variables y generando hipótesis sobre su relación con la variable objetivo ('Sobrevivió'). Luego, probaremos esas hipótesis a través de un conjunto de herramientas de análisis de datos exploratorios. Como resultado, obtendremos una visión completa de las variables que deberían pertenecer a nuestro modelo de predicción.

Empecemos:

* **Id del Pasajero**. Esta es solo una identificación única de cada pasajero. No se espera que sea relevante para nuestro análisis.
* **Sobrevivió**. Variable objetivo. Hundirse o no hundirse es la cuestión de este ejercicio.
* **Clase P**. Esta es la clase de boleto. Según Karl Marx, esto debería afectar nuestra variable objetivo. La primera clase debería tener una tasa de supervivencia más alta.
* **Nombre**. Los nombres son una forma de etiquetado social, especialmente cuando van acompañados de un título. Como consecuencia, puede dar lugar a diferentes formas de tratamiento. Mantengamos un ojo en esto.
* **Sexo**. Siempre importante.
* **Edad**. Debería marcar la diferencia. Por ejemplo, los niños suelen ser evacuados primero en un desastre, para que podamos pensar en una solución en silencio... Bromeando, la verdadera razón por la que la 'Edad' importa es esta [una] (http://www.dailymail.co) .uk/sciencetech/article-1254788/Por qué-mujeres-niños-salvados-Titanic-Lusitania.html#ixzz54KETWEPr).
* **Esp.Sib**. Número de hermanos/cónyuges a bordo del Titanic. Diría que es más fácil sobrevivir si estás con tu familia que si viajas solo. [Trabajo en equipo](https://youtu.be/1qzzYrCTKuk)!
* **Parque**. Número de padres/hijos a bordo del Titanic. Debería jugar con 'SibSp'.
* **Boleto**. Este es el número de billete. A menos que tenga alguna información sobre lugares, no debería ser importante para propósitos de predicción.
* **Tarifa**. La misma lógica que 'Pclass'.
* **Cabina**. El número de cabina puede indicar dónde estaban las personas durante el desastre. No sería sorprendente que tuviera alguna influencia en las posibilidades de supervivencia, pero esta variable se excluyó debido al alto porcentaje de valores faltantes.
* **Embarcado**. Cuando sale el sol, sale para todos. No es de esperar que la gente que viene de Cherburgo tenga más mala suerte que la gente que viene de Southampton. A menos que haya algún efecto de segundo orden, [como negarse a huir para mantener su honor como hombre] (http://www.mindblowing-facts.org/2013/07/the-only-japanese-who-survived-the -titanic-perdió-su-trabajo-por-que-era-conocido-como-cobarde-en-japon-por-no-morir-con-los-otros-pasajeros/), yo diría que esta variable es no importante.

Ahora, paso a paso, realicemos nuestro análisis.

### 2.2.1. Pclass

Nuestra hipótesis es que cuanto mayor sea la clase, mayores serán las posibilidades de supervivencia. Esto significa que una persona que viaja en primera clase tiene más posibilidades de sobrevivir que una persona que viaja en segunda o tercera clase.

Para visualizar si existe una relación entre 'Pclass' y 'Survival', hagamos un gráfico de barras.

In [ ]:
# Plot
sns.barplot(df['Pclass'],df['Survived']);

Nuestra hipótesis es que cuanto mayor sea la clase, mayores serán las posibilidades de supervivencia. Esto significa que una persona que viaja en primera clase tiene más posibilidades de sobrevivir que una persona que viaja en segunda o tercera clase.

Para visualizar si existe una relación entre 'Pclass' y 'Survival', hagamos un gráfico de barras.

### 2.2.2. Name/Title

Nuestra suposición es que el título de las personas influye en cómo son tratadas. En nuestro caso, tenemos varios títulos, pero solo algunos de ellos son compartidos por un número importante de personas. Por ello, sería interesante que pudiéramos agrupar algunos de los títulos y simplificar nuestro análisis.

Analicemos el título y veamos si podemos encontrar una manera sensata de agruparlos. Luego, probamos nuestros nuevos grupos y, si funciona de manera aceptable, lo mantenemos. Por ahora, la optimización no será un objetivo. La atención se centra en conseguir algo que pueda mejorar nuestra situación actual.

In [ ]:
# Cuenta cuantas personas tienen cada uno de los títulos
df.groupby(['Title'])['PassengerId'].count()

De los resultados anteriores podemos ver que:

* Títulos como 'Maestro', 'Señorita', 'Señor' y 'Señora' aparecen varias veces. En consecuencia, no los agruparemos.
* Con respecto a Mme y Mlle, podemos ver [aquí](https://www.frenchtoday.com/blog/french-culture/madame-or-mademoiselle-a-delicate-question) que corresponden a las categorías Señora y Señorita , respectivamente. En consecuencia, los asignaremos a esos títulos.
* Finalmente, agruparemos todos los demás títulos en un nuevo título llamado 'Otro'. Luego, definiremos 'Título' como una característica categórica y la trazaremos para ver cómo se ve. Si se ve bien, procederemos con esta nueva categorización.

In [ ]:
# Mapa de títulos agregados:
titles_dict = {'Capt': 'Other',
               'Major': 'Other',
               'Jonkheer': 'Other',
               'Don': 'Other',
               'Sir': 'Other',
               'Dr': 'Other',
               'Rev': 'Other',
               'Countess': 'Other',
               'Dona': 'Other',
               'Mme': 'Mrs',
               'Mlle': 'Miss',
               'Ms': 'Miss',
               'Mr': 'Mr',
               'Mrs': 'Mrs',
               'Miss': 'Miss',
               'Master': 'Master',
               'Lady': 'Other'}

In [ ]:
# títulos de grupo
df['Title'] = df['Title'].map(titles_dict)
df['Title'].head()

In [ ]:
# Transformar en categórico
df['Title'] = pd.Categorical(df['Title'])
df.dtypes

In [ ]:
# Plot
sns.barplot(x='Title', y='Survived', data=df);

Como ya sabemos, el diagrama de barras nos muestra una estimación del valor medio (altura de cada rectángulo) y una indicación de la incertidumbre alrededor de esa tendencia central (barras de error).

Nuestros resultados sugieren que:
* Las personas con el título 'Señor' sobrevivieron menos que las personas con cualquier otro título.
* Los títulos con una tasa de supervivencia superior al 50% son los que corresponden a títulos femeninos (Miss o Mrs) o infantiles (Master).
* Nuestra nueva categoría, 'Otros', debería ser más discreta. Como podemos ver en la barra de error (línea negra), existe una incertidumbre significativa en torno al valor medio. Probablemente, uno de los problemas es que estamos mezclando títulos masculinos y femeninos en la categoría 'Otros'. Deberíamos proceder con un análisis más detallado para resolver esto. Además, la categoría 'Maestro' parece tener un problema similar. Por ahora, no haremos ningún cambio, pero mantendremos estas dos situaciones en mente para futuras mejoras de nuestro conjunto de datos.

### 2.2.3. Sex

El sexo es uno de los temas más discutidos en la historia de la humanidad. Hay varias perspectivas sobre el tema, pero debo confesar que las perspectivas de Freud tuvieron un impacto significativo en mí porque me han mostrado el tema en una nueva perspectiva. Lo nuevo de Freud es que asoció muchos comportamientos "normales" a los impulsos sexuales, casi hasta el punto de hacernos cuestionar todo lo que hacemos. Al final, Freud se dio cuenta de que no todo se trataba de sexo. Como dijo, 'a veces un cigarro es solo un cigarro' (Freud solía fumar cigarros).

Dejando los preámbulos a un lado, lo que realmente necesitamos saber es si a veces un cigarro es solo un cigarro o no. Ya tenemos algunas pistas de que, en Titanic, las mujeres tenían una mayor tasa de supervivencia. Pero, nada mejor que una trama para ver qué está pasando.

In [ ]:
# Transformar en categórico
df['Sex'] = pd.Categorical(df['Sex'])

In [ ]:
# Plot
sns.barplot(df['Sex'],df['Survived']);

Nuestra hipótesis parece correcta. El mundo pertenece a las mujeres, y el Titanic también.

### 2.2.4. Age

'Edad' es la siguiente variable en la lista. Nuestra hipótesis es que los niños son más propensos a sobrevivir, mientras que las personas en su vida adulta pueden tener una menor tasa de supervivencia. Personalmente, no tengo ninguna intuición especial sobre los ancianos, ya que son los más vulnerables. Esto puede jugar para ambos lados: o las personas ayudan a los ancianos porque son más vulnerables, o no pueden hacer frente a los desafíos que plantea el naufragio de un barco.

Llamemos al sospechoso habitual (gráfico de barras) para que nos ayude a comprender la situación.

In [ ]:
# Plot
plt.figure(figsize=(25,10))
sns.barplot(df['Age'],df['Survived'], ci=None)
plt.xticks(rotation=90);

Con un poco de creatividad, podemos decir que la trama tiene tres regiones:

1. Una región que va entre los 0 y los 15 años;
2. Uno entre 15 y 48 años;
3. Una última entre los 48 y los 80 años.

Sé que esta división es discutible, especialmente en lo que concierne a las dos últimas categorías. Sin embargo, el punto es que esta división de categorías se ajusta a lo que sabemos sobre la forma en que se organiza nuestra sociedad: niños, adultos y ancianos. Por ahora, procedamos de esta manera.

In [ ]:
# Plot
'''
Probablemente, hay una manera más fácil de hacer esta trama. Tuve un problema al usar
plt.axvspan porque los valores xmin y xmax no eran
siendo graficado correctamente. Por ejemplo, definiría xmax = 12 y solo
el área entre 0 y 7 estaría llena. Esto sucedía porque mi
El eje X no sigue una secuencia regular (0, 1, ..., n). Después de algún juicio
y error, noté que xmin y xmax se refieren a la cantidad de elementos en
la coordenada del eje X que debe rellenarse. En consecuencia, definí dos
variables, x_limit_1 y x_limit_2, que cuentan el número de elementos que
debe completarse en cada intervalo. ¿Suena confuso? Para mi también.
'''
limit_1 = 12
limit_2 = 50

x_limit_1 = np.size(df[df['Age'] < limit_1]['Age'].unique())
x_limit_2 = np.size(df[df['Age'] < limit_2]['Age'].unique())

plt.figure(figsize=(25,10))
sns.barplot(df['Age'],df['Survived'], ci=None)

plt.axvspan(-1, x_limit_1, alpha=0.25, color='green')
plt.axvspan(x_limit_1, x_limit_2, alpha=0.25, color='red')
plt.axvspan(x_limit_2, 100, alpha=0.25, color='yellow')

plt.xticks(rotation=90);

In [ ]:
# Bin data
df['Age'] = pd.cut(df['Age'], bins=[0, 12, 50, 200], labels=['Child','Adult','Elder'])
df['Age'].head()

In [ ]:
# Plot
sns.barplot(df['Age'], df['Survived']);

La gráfica muestra que los niños tienen una mayor tasa de supervivencia. También muestra que, en términos de supervivencia, no hay una diferencia significativa entre las categorías 'Adulto' y 'Anciano'. Por ahora, no haremos ningún cambio porque hay una justificación teórica detrás de esta categorización. Sin embargo, parece que bastaría con distinguir entre niños y adultos.

### 2.2.5. FamilySize

En cuanto al tamaño de la familia, nuestra hipótesis es que aquellos que viajan solos, tienen una menor tasa de supervivencia. La idea es que las personas en familia puedan colaborar y ayudarse a escapar.

Veamos si eso tiene sentido usando nuestro [hermoso y único amigo] (https://youtu.be/LsQtnBu3p7Y), el gráfico de barras.

In [ ]:
# Plot
sns.barplot(df['FamilySize'], df['Survived']);

Como podemos ver, cuando 'FamilySize' está entre 0 y 3, nuestra hipótesis encuentra algo de apoyo. Las personas que viajan solas tienen una tasa de supervivencia más baja que las personas que viajan con una, dos o tres personas más.

Sin embargo, cuando FamilySize está entre 4 y 10, las cosas comienzan a cambiar. A pesar de la gran variabilidad de los resultados, la tasa de supervivencia desciende. Esto puede sugerir que nuestra hipótesis debe revisarse cuando 'FamilySize' es mayor que 3.

Esta variable parece ser más compleja de lo esperado. Por tanto, no haremos ninguna transformación en esta variable y la dejaremos como variable continua para conservar toda la información que tiene.

### 2.2.6. Fare

La misma lógica aplicada a 'Pclass' debería funcionar para 'Tarifa': tarifas más altas, mayor tasa de supervivencia.

Dado que ahora queremos establecer comparaciones entre diferentes niveles de una variable categórica, usaremos un gráfico de caja en lugar de un gráfico de barras.

In [ ]:
# Plot
plt.figure(figsize=(7.5,5))
sns.boxplot(df['Survived'], df['Fare']);

La trama sugiere que los que sobrevivieron pagaron una tarifa más alta. Como creemos que esta variable está conectada con 'Pclass', veamos cómo funcionan juntas.

In [ ]:
# Plot
sns.barplot(df['Survived'], df['Fare'], df['Pclass']);

Aquí tenemos un resultado interesante. Parece que la 'Tarifa' no marca la diferencia, en términos de supervivencia, si viaja en segunda o tercera clase. Sin embargo, si viaja en primera clase, cuanto mayor sea la tarifa, mayores serán las posibilidades de supervivencia. Teniendo esto en cuenta, tendría sentido crear funciones de interacción entre 'Tarifa' y 'Pclass'.

### 2.2.7. embarcado

La hipótesis sobre 'Embarcado' es que no influye en las posibilidades de supervivencia. Es difícil imaginar un escenario en el que la gente de Southampton, por ejemplo, tuviera una ventaja competitiva tal que los hiciera más aptos para sobrevivir que la gente de Queensland. Sí, en [Darwin](https://en.wikipedia.org/wiki/Natural_selection) creemos.

Una simple trama puede iluminarnos.

In [ ]:
# Plot
sns.barplot(df['Embarked'], df['Survived']);

Ups... Parece que las personas que se embarcan en C fueron seleccionadas por una entidad superior para sobrevivir. Esto es extraño y puede estar ocultando alguna relación que no es obvia con esta trama (por ejemplo, las personas que se embarcaron en C eran en su mayoría mujeres).

Profundicemos más.

In [ ]:
# Comparar con otras variables
df.groupby(['Embarked']).mean()

Parece que las personas que se embarcan en C pagaban más y viajaban en mejor clase que las personas que se embarcaban en Q y S.

In [ ]:
# Relación con la edad
df.groupby(['Embarked','Age'])['PassengerId'].count()

No significant differences can be found.

In [ ]:
# Relación con el sexo
df.groupby(['Embarked','Sex'])['PassengerId'].count()

No se pueden encontrar diferencias significativas.

Teniendo en cuenta los resultados anteriores, me siento tentado a decir que el punto de embarque no influye en la tasa de supervivencia. Lo que realmente parece estar influyendo es la clase en la que viajaba la gente y cuánto gastaba.

Por ahora, no eliminaré la variable porque siento que soy un poco parcial y trato de forzar una conclusión. Sin embargo, tengamos en cuenta que tal vez 'Embarcado' no afecte a 'Sobrevivido'.

## 2.3. Extracción de características

En el libro ['How Google Works'](https://amzn.to/2M9hvv7), Eric Schmidt y Jonathan Rosenberg refieren que el ingrediente secreto de Google es la 'percepción técnica'. Según los autores, es el conocimiento técnico fundamental que permite a las empresas crear excelentes productos, que brindan un valor real a los clientes. Por ejemplo, [PageRank](http://www.cs.princeton.edu/~chazelle/courses/BIB/pagerank.htm) dio una ventaja competitiva increíble a Google en relación con otros motores de búsqueda, al proporcionar una forma mucho mejor para clasificar los resultados de búsqueda en la web.
​
La extracción de características es nuestra visión tecnológica en el aprendizaje automático. Aborda el problema de lograr el conjunto de funciones más informativo y compacto para mejorar el rendimiento de los modelos de aprendizaje automático. Vayamos paso a paso. En primer lugar, estamos hablando de 'informativo'. Esto significa que estamos buscando características que puedan caracterizar el comportamiento de lo que estamos tratando de modelar. Por ejemplo, si queremos modelar el clima, características como la temperatura, la humedad y el viento son informativas (están relacionadas con el problema). Por el contrario, el resultado de un partido de fútbol no será una característica informativa porque no afecta el clima.
​
Con respecto a 'compacto', lo que queremos decir es que queremos excluir características irrelevantes de nuestro modelo. Hay varias razones para excluir características irrelevantes. En nuestro caso, diría que lo más importante es reducir el sobreajuste. Volviendo al ejemplo del clima: sabemos que los puntajes de fútbol no afectan el clima, pero supongamos que todos los casos de lluvia en nuestro conjunto de entrenamiento ocurren después de una victoria del [Benfica](https://youtu.be/qX0D7xkF33s). Entonces, nuestro modelo podría aprender que la lluvia está relacionada con las victorias del Benfica, lo cual no es cierto. Tal generalización incorrecta a partir de una característica irrelevante del conjunto de entrenamiento daría como resultado un modelo de aprendizaje automático que se ajusta a un conjunto particular de datos, pero no puede predecir las observaciones futuras de manera confiable (sobreajuste).
​
Estas dos cuestiones principales se abordan en las siguientes subsecciones:
1. Ingeniería de características, que está relacionada con la generación de características informativas;
2. Selección de funciones, que se refiere a la elección de un conjunto compacto de funciones.

### 2.3.1. Ingeniería de características

La ingeniería de características es el arte de convertir datos sin procesar en características útiles. Hay varias técnicas de ingeniería de funciones que puede aplicar para ser un artista. [Heaton (2016)](https://arxiv.org/pdf/1701.07852.pdf) presenta una lista completa de ellos. Usaremos solo dos técnicas:
* Transformaciones de Box-Cox [(Box & Cox 1964)](https://www.nuffield.ox.ac.uk/users/cox/cox72.pdf);
* Generación de polinomios mediante desarrollos no lineales.

Antes de la aplicación de estas técnicas, solo haremos algunos ajustes a los datos para prepararlos para el proceso de modelado.

#### Preparación de datos

In [ ]:
# Overview
df.head()

In [ ]:
# Drop feature
df.drop('PassengerId', axis=1, inplace=True)

In [ ]:
# Check features type
df.dtypes

In [ ]:
# Transform object into categorical
df['Embarked'] = pd.Categorical(df['Embarked'])
df['Pclass'] = pd.Categorical(df['Pclass'])
df.dtypes

In [ ]:
# Transform categorical features into dummy variables
df = pd.get_dummies(df, drop_first=1)  
df.head()

In [ ]:
# Get training and test sets
from sklearn.model_selection import train_test_split

X = df[df.loc[:, df.columns != 'Survived'].columns]
y = df['Survived']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=0)

#### Transformaciones de Box-Cox

Las transformaciones de Box-Cox tienen como objetivo normalizar las variables. Estas transformaciones son una alternativa a las transformaciones típicas, como las transformaciones de raíz cuadrada, las transformaciones logarítmicas y las transformaciones inversas. La principal ventaja de las transformaciones de Box-Cox es que normalizan de manera óptima la variable elegida. Por lo tanto, evitan la necesidad de probar aleatoriamente diferentes transformaciones y automatizan el proceso de transformación de datos.

In [ ]:
# Aplicar la transformación de Box-Cox
from scipy.stats import boxcox

X_train_transformed = X_train.copy()
X_train_transformed['Fare'] = boxcox(X_train_transformed['Fare'] + 1)[0]
X_test_transformed = X_test.copy()
X_test_transformed['Fare'] = boxcox(X_test_transformed['Fare'] + 1)[0]

#### Polynomials

Una forma estándar de enriquecer nuestro conjunto de características es generar polinomios. La expansión polinomial crea interacciones entre características, así como crea potencias (por ejemplo, el cuadrado de una característica). De esta forma introducimos una dimensión no lineal a nuestro conjunto de datos, lo que puede mejorar el poder predictivo de nuestro modelo.

Debemos escalar nuestras características cuando tenemos polinomios o términos de interacción en nuestro modelo. Estos términos tienden a producir [multicolinealidad](https://en.wikipedia.org/wiki/Multicollinearity), lo que puede hacer que nuestras estimaciones sean muy sensibles a cambios menores en el modelo. Escalar características a un rango nos permite reducir la multicolinealidad y sus problemas.

Para escalar las características, transformaremos los datos para que se encuentren entre un valor mínimo y máximo dado. Seguiremos la práctica común y diremos que nuestro valor mínimo es cero y nuestro valor máximo es uno.

In [ ]:
# Reescalar datos
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_train_transformed_scaled = scaler.fit_transform(X_train_transformed)
X_test_transformed_scaled = scaler.transform(X_test_transformed)

In [ ]:
# Obtener características polinómicas
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(degree=2).fit(X_train_transformed)
X_train_poly = poly.transform(X_train_transformed_scaled)
X_test_poly = poly.transform(X_test_transformed_scaled)

In [ ]:
# Debug
print(poly.get_feature_names())

### 2.3.2. Selección de características

El siguiente paso es realizar la selección de características. La selección de características se trata de elegir la información relevante. Es bueno agregar y generar funciones, pero en algún momento debemos excluir funciones irrelevantes. De lo contrario, estaremos penalizando el poder predictivo de nuestro modelo. Puede encontrar una introducción concisa al tema de la selección de funciones en [Guyon & Elisseeff (2003)] (http://www.jmlr.org/papers/volume3/guyon03a/guyon03a.pdf).

En este trabajo, utilizaremos un enfoque estadístico univariante. Este enfoque selecciona características en función de pruebas estadísticas univariadas entre cada característica y la variable de destino. La intuición es que las características que son independientes de la variable objetivo son irrelevantes para la clasificación.

Usaremos la prueba de chi-cuadrado para la selección de características. Esto significa que tenemos que elegir el número de características que queremos en el modelo. Por ejemplo, si queremos tener tres características en nuestro modelo, el método seleccionará las tres características con la puntuación $\chi^2$ más alta.

Dado que no conocemos el número ideal de funciones, probaremos el método con todas las funciones posibles y elegiremos la cantidad de funciones con mejor rendimiento.

#### Univariate statistics

In [ ]:
# Seleccione características usando la prueba de chi-cuadrado
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

## Obtener puntaje usando el modelo original
logreg = LogisticRegression(C=1)
logreg.fit(X_train, y_train)
scores = cross_val_score(logreg, X_train, y_train, cv=10)
print('CV accuracy (original): %.3f +/- %.3f' % (np.mean(scores), np.std(scores)))
highest_score = np.mean(scores)

## Obtenga puntaje usando modelos con selección de características
for i in range(1, X_train_poly.shape[1]+1, 1):
    # Select i features
    select = SelectKBest(score_func=chi2, k=i)
    select.fit(X_train_poly, y_train)
    X_train_poly_selected = select.transform(X_train_poly)

    # Modelo con características i seleccionadas
    logreg.fit(X_train_poly_selected, y_train)
    scores = cross_val_score(logreg, X_train_poly_selected, y_train, cv=10)
    print('CV accuracy (number of features = %i): %.3f +/- %.3f' % (i, 
                                                                     np.mean(scores), 
                                                                     np.std(scores)))
    
    # Guardar resultados si la mejor puntuación
    if np.mean(scores) > highest_score:
        highest_score = np.mean(scores)
        std = np.std(scores)
        k_features_highest_score = i
    elif np.mean(scores) == highest_score:
        if np.std(scores) < std:
            highest_score = np.mean(scores)
            std = np.std(scores)
            k_features_highest_score = i
        
# Imprimir el número de características
print('Number of features when highest score: %i' % k_features_highest_score)

---

# 3. Modelo de unicornio

Las empresas emergentes utilizan el término "unicornio" para describir una empresa emergente valorada en mil millones de dólares o más. Independientemente de si se trata de [mil millones europeos o estadounidenses] (http://mathforum.org/library/drmath/view/52579.html), mil millones es un número grande. De hecho, es tan grande y raro que cuando encontramos startups con tanto valor, las asociamos a esas criaturas míticas que son los unicornios.

Hemos recorrido un largo camino desde que empezamos a resolver este problema. Desde nuestro comienzo con un modelo lean, hemos estado escalando nuestra puesta en marcha: imputamos datos faltantes, realizamos un análisis exploratorio de datos y extrajimos características. También tuvimos que lidiar con bromas terribles sobre el Titanic que tardan un tiempo en asimilarse.

Ahora es el momento de convertir todo este trabajo en un modelo de alta precisión, nuestro modelo 'unicornio'.

## 3.1. Ajuste el modelo para la mejor combinación de características

In [ ]:
# Seleccionar características
select = SelectKBest(score_func=chi2, k=k_features_highest_score)
select.fit(X_train_poly, y_train)
X_train_poly_selected = select.transform(X_train_poly)

In [ ]:
# Modelo de ajuste
logreg = LogisticRegression(C=1)
logreg.fit(X_train_poly_selected, y_train)

In [ ]:
# Rendimiento del modelo
scores = cross_val_score(logreg, X_train_poly_selected, y_train, cv=10)
print('CV accuracy: %.3f +/- %.3f' % (np.mean(scores), np.std(scores)))

## 3.2. Curva de aprendizaje

In [ ]:
# Trazar curvas de aprendizaje
title = "Learning Curves (Logistic Regression)"
cv = 10
plot_learning_curve(logreg, title, X_train_poly_selected, 
                    y_train, ylim=(0.7, 1.01), cv=cv, n_jobs=1);

Sin signos de sobreajuste o desajuste.

## 3.3. Validation curve

In [ ]:
# Trazar la curva de validación
title = 'Validation Curve (Logistic Regression)'
param_name = 'C'
param_range = [0.001, 0.01, 0.1, 1.0, 10.0, 100.0] 
cv = 10
plot_validation_curve(estimator=logreg, title=title, X=X_train_poly_selected, y=y_train, 
                      param_name=param_name, ylim=(0.5, 1.01), param_range=param_range);

Usamos C=1, que está al borde del sobreajuste. En cuanto al underfitting, no hay indicios de ello ya que el modelo funciona bien.

## 3.4. Enviar predicciones

In [ ]:
# Obtener conjunto de datos de prueba
df = pd.read_csv('../input/test.csv')
df_raw = df.copy()

In [ ]:
# Transformar conjunto de datos (basado en el Capítulo 2)

## 2.2
df['FamilySize'] = df['SibSp'] + df['Parch']
df.drop('SibSp',axis=1,inplace=True)
df.drop('Parch',axis=1,inplace=True)
df.drop(['Name','Ticket','Cabin'], axis=1, inplace=True)

df['Title']=0
for i in df:
    df['Title']=df_raw['Name'].str.extract('([A-Za-z]+)\.', expand=False)  
df_raw['Title'] = df['Title']  
means = df_raw.groupby('Title')['Age'].mean()
map_means = means.to_dict()
idx_nan_age = df.loc[np.isnan(df['Age'])].index
df.loc[idx_nan_age, 'Age'] = df['Title'].loc[idx_nan_age].map(map_means)
df['Title'] = df['Title'].map(titles_dict)
df['Title'] = pd.Categorical(df['Title'])

df['Imputed'] = 0
df.at[idx_nan_age.values, 'Imputed'] = 1

df['Age'] = pd.cut(df['Age'], bins=[0, 12, 50, 200], labels=['Child','Adult','Elder'])

## 2.3
passenger_id = df['PassengerId'].values
df.drop('PassengerId', axis=1, inplace=True)
df['Embarked'] = pd.Categorical(df['Embarked'])
df['Pclass'] = pd.Categorical(df['Pclass'])
df = pd.get_dummies(df, drop_first=1)

df = df.fillna(df.mean())  # Falta un valor en 'Tarifa'

X = df[df.loc[:, df.columns != 'Survived'].columns]

X_transformed = X.copy()
X_transformed['Fare'] = boxcox(X_transformed['Fare'] + 1)[0]

scaler = MinMaxScaler()
X_transformed_scaled = scaler.fit_transform(X_transformed)

poly = PolynomialFeatures(degree=2).fit(X_transformed)
X_poly = poly.transform(X_transformed_scaled)

X_poly_selected = select.transform(X_poly)

In [ ]:
# Hacer predicciones
predictions = logreg.predict(X_poly_selected)

In [ ]:
# Generar archivo de envío
submission = pd.DataFrame({ 'PassengerId': passenger_id,
                            'Survived': predictions})
submission.to_csv("submission.csv", index=False)

---

# 4. Conclusión

Como [Halevy et al. (2009)](https://static.googleusercontent.com/media/research.google.com/pt-PT//pubs/archive/35179.pdf) señaló que "invariablemente, los modelos simples y una gran cantidad de datos triunfan sobre los más elaborados modelos basados en menos datos.' [Monica Rogati](https://youtu.be/F7iopLnhDik) agregó que "mejores datos vencen a más datos". Basado en estos principios, el objetivo de este estudio fue mejorar la calidad de los datos a través del análisis exploratorio de datos y la extracción de características. No usamos un algoritmo inteligente, pero exploramos técnicas inteligentes para mejorar nuestros datos.
​
Mi expectativa es que después de leer este kernel, pueda comenzar a compilar un libro de recetas de técnicas en análisis exploratorio de datos y extracción de características. Estas técnicas lo ayudarán a obtener confianza en sus datos y a comprometerse con cualquier problema de ciencia de datos. Además, cuanto más utilice y perfeccione estas técnicas, más desarrollará su intuición para resolver problemas.
​
Ahora es tu turno. Haz tuyo este trabajo. Seleccione una parte de este núcleo y juegue con él. ¿Por qué no probar un proceso de selección de características diferente? ¿O qué hay de aplicar un método de imputación diferente? Hay cien formas diferentes de [robar esta obra como un artista](https://youtu.be/oww7oB9rjgw). Hazlo... Después de todo, todos los unicornios comenzaron con un MVP.

---

# También podría gustarte
* [Mi blog](http://pmarcelino.com/)
* [Exploración completa de datos en Python](https://www.kaggle.com/pmarcelino/comprehensive-data-exploration-with-python)

# Referencias

**Libros**

* [Ries, E., 2011. The Lean Startup: cómo los empresarios de hoy usan la innovación continua para crear negocios radicalmente exitosos] (https://amzn.to/2JsIMH3)
* [Hair, J.F., Black, W.C., Babin, B.J., Anderson, R.E. y Tatham, R.L., 2013. Análisis de datos multivariante](https://amzn.to/2JtC1Vm)
* [Asefeso, A., Lund, S.B., Parry, H., 2014. Mantenga sus ojos en el horizonte: Lecciones de negocios del insumergible Titanic](https://amzn.to/2JwpsIT)

**Documentos**

* [Heaton, J., 2016. Un análisis empírico de la ingeniería de características para el modelado predictivo. En SoutheastCon, 2016 (págs. 1-6). IEEE.](https://arxiv.org/pdf/1701.07852.pdf)
* [Recuadro, G.E. y Cox, D.R., 1964. Un análisis de las transformaciones. Revista de la Real Sociedad Estadística. Serie B (Metodológica), pp.211-252.](https://www.nuffield.ox.ac.uk/users/cox/cox72.pdf)
* [Guyon, I. y Elisseeff, A., 2003. Una introducción a la selección de características y variables. Diario de investigación de aprendizaje automático, 3 (marzo), pp.1157-1182.] (http://www.jmlr.org/papers/volume3/guyon03a/guyon03a.pdf)
* [Halevy, A., Norvig, P. y Pereira, F., 2009. La irrazonable efectividad de los datos. IEEE Intelligent Systems, 24(2), pp.8-12.](https://static.googleusercontent.com/media/research.google.com/pt-PT//pubs/archive/35179.pdf)

**Vídeos**

* [Rogati, M., 2012. El modelo y el tren descarrilado: un procedimiento de datos de entrenamiento. Estratos de O'Reilly.](https://youtu.be/F7iopLnhDik)
* [Kleon, A., 2012. Roba como un artista. Charlas TEDx.](https://youtu.be/oww7oB9rjgw)